In [2]:
!pip install BTrees


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 7.5 MB/s eta 0:00:00


In [9]:
import csv
import timeit
from BTrees.OOBTree import OOBTree
from collections import defaultdict


In [10]:
tree_by_price = OOBTree()
dict_data = {}

def add_item_to_tree(tree, item):
    price = float(item["Price"])
    if price not in tree:
        tree[price] = []
    tree[price].append({
        "ID": item["ID"],
        "Name": item["Name"],
        "Category": item["Category"]
    })

def add_item_to_dict(d, item):
    d[item["ID"]] = {
        "Name": item["Name"],
        "Category": item["Category"],
        "Price": float(item["Price"])
    }


In [11]:
tree = OOBTree()
dict_data = {}

with open("/content/sample_data/generated_items_data.csv", newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        add_item_to_tree(tree_by_price, row)
        add_item_to_dict(dict_data, row)

print(f"✅ Завантажено товарів: {len(dict_data)}")


✅ Завантажено товарів: 100000


In [12]:
def range_query_tree(tree, min_price, max_price):
    result = []
    for price, items in tree.items(min_price, max_price):
        result.extend(items)
    return result

def range_query_dict(d, min_price, max_price):
    return [
        v for v in d.values()
        if min_price <= v["Price"] <= max_price
    ]


In [13]:
min_price = 50
max_price = 100

# 100 запитів до OOBTree
tree_time = timeit.timeit(lambda: range_query_tree(tree_by_price, min_price, max_price), number=100)

# 100 запитів до dict
dict_time = timeit.timeit(lambda: range_query_dict(dict_data, min_price, max_price), number=100)

# Вивід результатів
print(f"Total range_query time for OOBTree: {tree_time:.6f} seconds")
print(f"Total range_query time for Dict:    {dict_time:.6f} seconds")


Total range_query time for OOBTree: 0.100760 seconds
Total range_query time for Dict:    3.651136 seconds


У ході виконання завдання було реалізовано дві структури для зберігання товарів — OOBTree (з ключем Price) та стандартний dict (з ключем ID). Для кожної структури було реалізовано діапазонні запити з метою пошуку товарів у заданому ціновому діапазоні.

Проведене тестування показало:

  - OOBTree значно ефективніший при виконанні діапазонних запитів завдяки використанню методу .items(min_price, max_price), який забезпечує доступ лише до релевантної частини даних.

  - Стандартний dict, який не підтримує індексацію, виконує діапазонний запит через повний лінійний перебір усіх елементів, що значно знижує продуктивність при великих обсягах даних.

  - Замір часу для 100 запитів підтвердив очікуване: OOBTree продемонстрував швидший час виконання у порівнянні з dict.

Таким чином, OOBTree є кращим вибором для задач, де критичне значення має ефективне виконання діапазонних запитів за числовим ключем.